In [14]:
import os
import sys
from pathlib import Path
from glob import glob
import json


currentdir = Path.cwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 
sys.path.insert(1, os.path.join(parentdir, 'evaluation')) 

from evaluation import metrics_csvreader

In [2]:
csvfile = "/work/mi186veva-results/MainExperiment_3d_3_and_5/3D3_Ns_WT_Sm_S5_BT_B0.15_B0.7_/Results_with_EarlyStopping/metrics_by_threshold_0.364.csv"
comparison = "/work/mi186veva-results/MainExperiment_3d_3_and_5/3D3_Ns_WT_Sm_S5_BT_B0.15_B0.7_/Results_with_EarlyStopping/compare_all_thresholds.csv"

metrics = metrics_csvreader.MetricsCsvReaderComparisonFiles().read_metrics(comparison)

In [18]:


folders = glob("/work/mi186veva-results/MainExperiment_3d_3_and_5/*/")

print(len(folders))

groups = {}

for folder in folders:
    name = folder.split("/")[-2]
    csvfile = os.path.join(folder, "Results_with_EarlyStopping", "metrics_by_threshold_0.364.csv")
    metrics = metrics_csvreader.MetricsCsvReader().read_metrics(csvfile)
    
    value = str(metrics["Accuracy"]["mean"])
    if not str(value) in groups:
        groups[value] = []
    groups[value].append(name)
    
print(json.dumps(groups, indent = 4))

249
{
    "0.1024843258534392": [
        "3D3_Ns_WF_Sm_S3_BT_B0.1_B0.7_",
        "3D3_Ns_WF_Sm_S7_BT_B0.15_B0.4_",
        "3D3_Ns_WF_Sg_S1_BT_B0.1_B0.4_",
        "3D3_Ns_WF_Sg_S1.5_BT_B0.15_B0.7_",
        "3D3_Ns_WF_S_S-1_BT_B0.1_B0.7_",
        "3D3_Ns_WF_S_S-1_BT_B0.15_B0.25_",
        "3D3_Ns_WF_Sm_S3_BT_B0.15_B0.6_",
        "3D3_Ns_WF_Sm_S3_BT_B0.15_B0.25_",
        "3D3_Ns_WF_S_S-1_BF_B0.1_B0.25_",
        "3D3_Ns_WF_Sg_S0.5_BT_B0.15_B0.7_",
        "3D3_Ns_WF_Sm_S7_BT_B0.1_B0.4_",
        "3D3_Ns_WF_Sg_S1_BT_B0.15_B0.6_",
        "3D3_Ns_WF_Sg_S0.5_BT_B0.15_B0.25_",
        "3D3_Ns_WF_Sm_S3_BT_B0.1_B0.25_",
        "3D3_Ns_WF_Sm_S5_BF_B0.1_B0.25_",
        "3D3_Ns_WF_Sg_S1.5_BT_B0.1_B0.4_",
        "3D3_Ns_WF_Sg_S0.5_BT_B0.15_B0.6_",
        "3D3_Ns_WF_Sg_S1_BT_B0.15_B0.7_",
        "3D3_Ns_WF_Sm_S5_BT_B0.15_B0.4_",
        "3D3_Ns_WF_Sm_S3_BT_B0.1_B0.6_",
        "3D3_Ns_WF_S_S-1_BT_B0.1_B0.6_",
        "3D3_Ns_WF_Sg_S1.5_BT_B0.15_B0.6_",
        "3D3_Ns_WF_Sm_S3_BT_B0.15_